In [2]:
import itertools

# Our numerical workhorses
import numpy as np
import pandas as pd
import scipy.integrate

# Import Altair for high level plotting
import altair as alt
import altair_catplot as altcat

# Import Bokeh modules for interactive plotting
import bokeh.io
import bokeh.plotting

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

# Pevent bulky altair plots
alt.data_transformers.enable('json')

Loading BokehJS ...

DataTransformerRegistry.enable('json')

In [14]:
fname = '../data/fish_activity_for_validation/150717_2A_2B.csv'
df = pd.read_csv(fname, comment='#')

Notes: 
1. middur columns is what we use for computing activity.
2. Only the fish in instrument 2A, numbered 1 through 96 in the activity data file 150717_2A_2B.csv, were genotyped. The fish in instrument 2B, numbered 97 and above, are not used in the assay. **does this mean we filter them out?**

Enforce that fish 1-96 are here


In [15]:
df.head(10)

,location,animal,user,sn,an,datatype,start,end,startreason,endreason,frect,fredur,midct,middur,burct,burdur,stdate,sttime
0,c97,z097,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,34,55.7,34,4.3,0,0.0,17/07/2015,14:30:00
1,c98,z098,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,0,60.0,0,0.0,0,0.0,17/07/2015,14:30:00
2,c99,z099,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,6,59.0,6,1.0,0,0.0,17/07/2015,14:30:00
3,c100,z100,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,25,55.8,25,4.2,0,0.0,17/07/2015,14:30:00
4,c101,z101,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,1,60.0,0,0.0,0,0.0,17/07/2015,14:30:00
5,c102,z102,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,0,60.0,0,0.0,0,0.0,17/07/2015,14:30:00
6,c103,z103,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,10,58.3,10,1.7,0,0.0,17/07/2015,14:30:00
7,c104,z104,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,38,53.4,39,6.6,0,0.0,17/07/2015,14:30:00
8,c105,z105,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,0,60.0,0,0.0,0,0.0,17/07/2015,14:30:00
9,c106,z106,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,21,56.4,22,3.6,1,0.1,17/07/2015,14:30:00


In [9]:
txt = open("../data/fish_activity_for_validation/150717_2A_genotype_3.txt", "r")

In [10]:
txt.read()

'wt\thet\tmut\n1\t5\t2\n3\t6\t4\n8\t7\t14\n12\t9\t16\n21\t10\t26\n28\t13\t27\n31\t15\t37\n33\t17\t40\n34\t18\t41\n35\t19\t46\n38\t20\t54\n39\t23\t57\n42\t24\t58\n43\t25\t60\n53\t29\t71\n61\t30\t75\n73\t32\t76\n79\t44\t77\n87\t45\t81\n88\t48\t82\n93\t49\t\n\t50\t\n\t52\t\n\t56\t\n\t59\t\n\t63\t\n\t72\t\n\t74\t\n\t78\t\n\t80\t\n\t85\t\n\t86\t\n\t89\t\n\t90\t\n\t91\t\n\t92\t\n\t94\t\n\t96\t'

In [20]:
def test_column_names(df, fname):
    """Ensure DataFrame has proper columns."""
    column_names = ['location', 'animal', 'user', 'sn', 'an', 'datatype', 'start',\
                    'end', 'startreason', 'endreason','frect', 'midct', 'middur', \
                    'burct', 'burdur', 'stdate', 'sttime']

    assert list(df.columns) == column_names, fname + ' has wrong column names.'

In [21]:
test_column_names(df, fname)

AssertionError: ../data/fish_activity_for_validation/150717_2A_2B.csv has wrong column names.